In [17]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import pyeeg
import pickle
from tqdm import tqdm
import mne
import picard
import time
import sys
import logging
import statistics
import pywt
logging.getLogger("mne").setLevel(logging.WARNING)
#тут нужно указать путь в __init__.py из папки entropy

MODULE_PATH = "C:/Users/Никита/Desktop/entropy/__init__.py"

MODULE_NAME = "entropy"

import importlib
import sys

spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)

from entropy import *

import warnings
warnings.filterwarnings('ignore')


In [42]:
#функции для извлечения фичей из сырого ээг - сигнала

def first_der(x):
    return np.mean(np.abs(x[1:]-x[0:-1]))

def second_der(x):
    return np.mean(np.abs(x[2:] - x[0:-2]))

def first_der_norm(x):
    return first_der(x/max(np.abs(x)))

def second_der_norm(x):
    return second_der(x/max(np.abs(x)))
def samp_entropy(X):
    return sample_entropy(X)

def spec_entropy(X):
    return spectral_entropy(X, 100)

def sing_entropy(X):
    return svd_entropy(X)

def petrosyan(X):
    return petrosian_fd(X)

def Hig(X):  
    return higuchi_fd(X)

def Katz(X):
    return katz_fd(X)

def power_4_8(X):
    return pyeeg.bin_power(X, [4, 8], 1000)[0][0]

def power_8_16(X):
    return pyeeg.bin_power(X, [8, 16], 1000)[0][0]

def power_16_32(X):
    return pyeeg.bin_power(X, [16, 32], 1000)[0][0]

def power_32_64(X):
    return pyeeg.bin_power(X, [32, 64], 1000)[0][0]

def freq_delta(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    delta = frequency[L*1//Fs-1: L*4//Fs]
    return np.std(delta)

def freq_theta(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    theta = frequency[L*4//Fs-1: L*8//Fs]
    return np.std(theta)

def freq_alpha(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    alpha = frequency[L*5//Fs-1: L*13//Fs]
    return np.std(alpha)

def freq_beta(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    beta = frequency[L*13//Fs-1: L*30//Fs]
    return np.std(beta)

def freq_gamma(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    gamma = frequency[L*30//Fs-1: L*50//Fs]
    return np.std(gamma)

def wwlt(X):
    coeffs = pywt.wavedec(X, 'db1', level=5)
    cA2, cD2,cD3,cD4,cD5, cD6 = coeffs
    gamma = cD5
    beta = cD4
    alpha = cD3
    theta = cD2
    return sum(gamma**2), sum(beta**2), sum(alpha**2), sum(theta**2)



functions = [np.mean,
             #np.std,
             first_der,second_der,statistics.median,
             #first_der_norm, second_der_norm,perm_entropy,app_entropy, samp_entropy,
            #spec_entropy, sing_entropy, petrosyan, Hig, Katz, power_4_8, power_8_16, power_16_32, power_32_64,
            #freq_delta, freq_theta, freq_alpha,
             freq_beta, freq_gamma]
def remove_eye_activity_ICA(data):
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    ch_names = ["AF3", "F3", "F7", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F8", "F4","AF4"]
    EOG_chanels = [33, 34, 35, 36]
    eeg = []
    
    for ch in good_chanels:
        eeg.append(np.array(data[ch]))

    for ch in EOG_chanels:
        eeg.append(np.array(data[ch]))
        
    eeg = np.array(eeg) * 10e-6              #in volts
    
    info = mne.create_info(ch_names, 512, ch_types=["eeg"] * 14 + ["eog"] * 4)
    raw = mne.io.RawArray(eeg, info)
    
    raw_tmp = raw.copy()
    raw_tmp.filter(1, None)
    
    ica = mne.preprocessing.ICA(method="picard",
                            fit_params={"extended": True},
                            random_state=1)
    ica.fit(raw_tmp)
    #TODO: WHAT ICA'S ?
    #ica.exclude = [1]
    
    raw_corrected = raw.copy()
    
    return raw_corrected.get_data()

def remove_eye_activity_regr(data):
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    EOG_chanels = [33, 34, 35, 36]
    eeg = []
    for ch in good_chanels:
        eeg.append(np.array(data[ch]))

    for ch in EOG_chanels:
        eeg.append(np.array(data[ch]))
    eeg = np.array(eeg) * 10e-6
    eeg1 = eeg.copy()
    info = mne.create_info(18, 512, ch_types=["eeg"] * 14 + ["eog"] * 4)
    raw1 = mne.io.RawArray(eeg, info)
    raw2 = mne.io.RawArray(eeg1, info)
    bip = np.array([[-1, 1, 0, 0], [0, 0, -1, 1]])
    raw1_eog = bip @ raw1[14:, :][0]
    raw2_eog = bip @ raw2[14:, :][0]
    raw1_eeg = raw1[:14, :][0]
    raw2_eeg = raw2[:14, :][0]
    b = np.linalg.inv(raw1_eog @ raw1_eog.T) @ raw1_eog @ raw1_eeg.T
    eeg_corrected = (raw2_eeg.T - raw2_eog.T @ b).T
    raw3 = raw2.copy()
    raw3._data[:14, :] = eeg_corrected
    return np.array(raw3.to_data_frame().T[1:])

def create_df(data, lables,subject, cut=2500, train=True, n_segments=10):
    features_all = []
    m = len(data[0])
    #нужные каналы
    
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    ch_names = ["AF3", "F3", "F7", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F8", "F4","AF4"]
    #бьем нашу ээг на сегменты и бежим по ним в цикле 
    
    for j in range(n_segments):
        start = cut + j*(m - cut)//n_segments
        finish = cut + (j + 1)*(m - cut)//n_segments
        features = dict()
        for k in range(14):
            for func in functions:
                features[f"channel_{ch_names[k]}_{func.__name__}"] = func(data[k][start:finish])
        features['arousal'] = lables[0]
        features['valence'] = lables[1]
        features['dominance'] = lables[2]
        features['liking'] = lables[3]
        features['subject'] = subject
        features_all.append(features)
    return pd.json_normalize(features_all)


In [43]:
datasets = []

# бежим по дериктории, применяем извлечение фичей ко всем файлам и склеиваем все фичи в один датасет

for fn in tqdm(os.listdir('data_preprocessed_python/')):
    with open('data_preprocessed_python/'+fn, 'rb') as f:
        raw = pickle.loads(f.read(), encoding='latin1')
        video = 0
        while video < 40:
            label = raw['labels'][video]
            #print('Video:', video + 1,'file:', fn)
            datasets.append(create_df(raw['data'][video], label, fn.split('.')[0]))
            video += 1

data_all = pd.concat(datasets)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [01:06<00:00,  2.07s/it]


In [44]:
#wawelet_features = list(filter(lambda x: x.endswith('wwlt'), data_all.columns))

In [45]:

#for wfeature in wawelet_features:
#    for i in range(4):
#        data_all[wfeature + f"_{i}"] = data_all[wfeature].apply(lambda x: x[i])


'\nfor wfeature in wawelet_features:\n    for i in range(4):\n        data_all[wfeature + f"_{i}"] = data_all[wfeature].apply(lambda x: x[i])\n'

In [46]:
#data_all.drop(wawelet_features, axis=1, inplace=True)

In [47]:
#перемешиваем фичи, для улучшения обучения

data_all = data_all.sample(frac=1)
data_all
#data_all.to_excel('features_and_lables.xlsx', index=False)

,channel_AF3_mean,channel_AF3_first_der,channel_AF3_second_der,channel_AF3_median,channel_AF3_freq_beta,channel_AF3_freq_gamma,channel_F3_mean,channel_F3_first_der,channel_F3_second_der,channel_F3_median,...,channel_AF4_first_der,channel_AF4_second_der,channel_AF4_median,channel_AF4_freq_beta,channel_AF4_freq_gamma,arousal,valence,dominance,liking,subject
6,-0.338275,16.131208,23.180256,-0.129784,0.793932,0.780563,-0.081354,6.631184,9.215200,-0.205032,...,17.743119,25.465390,-0.200295,0.901211,0.845695,7.09,5.67,2.53,5.65,s16
6,0.160061,12.975685,18.772966,-0.122302,0.633733,0.622272,-0.003489,11.121554,17.020755,0.289265,...,3.293607,4.964798,-0.110546,0.169399,0.150099,5.31,2.88,3.85,5.04,s10
9,-0.128207,6.368925,10.999833,0.140995,0.450619,0.182991,0.036473,2.396017,4.014938,-0.012072,...,2.072972,3.485676,-0.146730,0.144524,0.053328,7.00,6.96,8.00,2.92,s05
6,-0.330680,6.984552,9.881468,-0.384654,0.342087,0.323896,-0.210430,4.056647,6.312295,-0.377778,...,6.744868,9.741950,-0.577637,0.386138,0.296712,7.09,5.03,4.97,7.12,s07
0,0.015647,9.312066,11.864346,0.141592,0.450701,0.536549,0.035081,6.599388,9.096703,0.285940,...,10.600693,14.155881,0.455511,0.545261,0.596937,9.00,5.13,8.01,8.95,s15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0.085569,5.163994,7.706871,0.221582,0.357493,0.162925,0.044349,3.472104,5.461471,0.312398,...,3.867114,6.280913,-0.127556,0.367121,0.138292,7.05,5.06,7.03,6.97,s30
7,0.041417,9.289765,13.479939,0.323124,0.576428,0.444103,0.134033,7.645286,10.819611,0.462295,...,3.083855,4.981682,-0.270975,0.175043,0.099426,4.21,5.06,5.00,6.24,s08
4,-0.099369,7.432529,10.650278,-0.119078,0.467868,0.335091,-0.086763,8.751743,13.751471,-0.518359,...,8.358198,12.891586,1.859763,0.336646,0.411864,8.82,6.40,6.31,8.53,s08
4,-0.159367,4.662700,7.334826,-0.011366,0.268488,0.173939,0.041676,3.727110,5.796741,0.085752,...,4.397370,7.036674,0.161387,0.243761,0.132082,3.87,3.09,3.06,6.36,s14


In [48]:
# получаем X и Y для обучения

X = data_all.drop(['arousal', 'valence', 'dominance', 'liking', 'subject'], axis=1)
y = data_all['arousal'].apply(lambda x: 0 if x <= 5.5 else 1)
                              #if x > 4.5 and x < 5.5 else 2)

In [49]:
#сама модель

from catboost import Pool, cv


cv_dataset = Pool(data=X,
                  label=y)

params = {"iterations": 10000,
          "depth": 6,
          "loss_function": "Logloss",
          "custom_metric": "Accuracy",
          "verbose": False}

scores = cv(cv_dataset,
            params,
            fold_count=3,
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [33]:
from catboost import CatBoostClassifier

cat_arousal_model = CatBoostClassifier(iterations=10000, depth=6)
cat_dominance_model = CatBoostClassifier(iterations=10000, depth=6)
cat_valence_model = CatBoostClassifier(iterations=10000, depth=6)

In [34]:
def train_model(measure, data, model):
    X = data_all.drop(['arousal', 'valence', 'dominance', 'liking', 'subject'], axis=1)
    y = data_all[measure].apply(lambda x: 0 if x <= 4.5 else 1 if x > 4.5 and x < 5.5 else 2 )
    model.fit(X, y)
    return model

In [35]:
arousal_model = train_model('arousal', data_all, cat_arousal_model)


Learning rate set to 0.012876
0:	learn: 1.0963589	total: 73.6ms	remaining: 12m 15s
1:	learn: 1.0941359	total: 142ms	remaining: 11m 50s
2:	learn: 1.0917713	total: 202ms	remaining: 11m 13s
3:	learn: 1.0897047	total: 275ms	remaining: 11m 27s
4:	learn: 1.0875353	total: 338ms	remaining: 11m 14s
5:	learn: 1.0856873	total: 393ms	remaining: 10m 54s
6:	learn: 1.0837895	total: 459ms	remaining: 10m 55s
7:	learn: 1.0819659	total: 527ms	remaining: 10m 58s
8:	learn: 1.0800532	total: 585ms	remaining: 10m 49s
9:	learn: 1.0780770	total: 654ms	remaining: 10m 53s
10:	learn: 1.0759984	total: 718ms	remaining: 10m 51s
11:	learn: 1.0741043	total: 784ms	remaining: 10m 52s
12:	learn: 1.0722187	total: 851ms	remaining: 10m 53s
13:	learn: 1.0701025	total: 918ms	remaining: 10m 54s
14:	learn: 1.0683161	total: 991ms	remaining: 10m 59s
15:	learn: 1.0666945	total: 1.06s	remaining: 11m 3s
16:	learn: 1.0648339	total: 1.13s	remaining: 11m 6s
17:	learn: 1.0631141	total: 1.2s	remaining: 11m 6s
18:	learn: 1.0614964	total: 1

155:	learn: 0.9538878	total: 10.9s	remaining: 11m 30s
156:	learn: 0.9533732	total: 11s	remaining: 11m 29s
157:	learn: 0.9530877	total: 11.1s	remaining: 11m 28s
158:	learn: 0.9528085	total: 11.1s	remaining: 11m 28s
159:	learn: 0.9523453	total: 11.2s	remaining: 11m 28s
160:	learn: 0.9519105	total: 11.2s	remaining: 11m 27s
161:	learn: 0.9515934	total: 11.3s	remaining: 11m 26s
162:	learn: 0.9512910	total: 11.4s	remaining: 11m 25s
163:	learn: 0.9509228	total: 11.4s	remaining: 11m 24s
164:	learn: 0.9505227	total: 11.5s	remaining: 11m 23s
165:	learn: 0.9503205	total: 11.5s	remaining: 11m 22s
166:	learn: 0.9499299	total: 11.6s	remaining: 11m 21s
167:	learn: 0.9497035	total: 11.6s	remaining: 11m 21s
168:	learn: 0.9494369	total: 11.7s	remaining: 11m 21s
169:	learn: 0.9490309	total: 11.8s	remaining: 11m 22s
170:	learn: 0.9487677	total: 11.9s	remaining: 11m 22s
171:	learn: 0.9484072	total: 12s	remaining: 11m 22s
172:	learn: 0.9481616	total: 12s	remaining: 11m 23s
173:	learn: 0.9478287	total: 12.1s

309:	learn: 0.9091219	total: 20.4s	remaining: 10m 36s
310:	learn: 0.9089048	total: 20.4s	remaining: 10m 35s
311:	learn: 0.9085387	total: 20.5s	remaining: 10m 35s
312:	learn: 0.9082401	total: 20.5s	remaining: 10m 35s
313:	learn: 0.9081217	total: 20.6s	remaining: 10m 35s
314:	learn: 0.9079338	total: 20.7s	remaining: 10m 35s
315:	learn: 0.9077256	total: 20.7s	remaining: 10m 35s
316:	learn: 0.9073902	total: 20.8s	remaining: 10m 35s
317:	learn: 0.9071610	total: 20.9s	remaining: 10m 35s
318:	learn: 0.9069549	total: 20.9s	remaining: 10m 35s
319:	learn: 0.9067442	total: 21s	remaining: 10m 35s
320:	learn: 0.9065436	total: 21.1s	remaining: 10m 34s
321:	learn: 0.9062381	total: 21.1s	remaining: 10m 34s
322:	learn: 0.9060974	total: 21.2s	remaining: 10m 33s
323:	learn: 0.9059602	total: 21.2s	remaining: 10m 33s
324:	learn: 0.9057854	total: 21.3s	remaining: 10m 33s
325:	learn: 0.9055589	total: 21.3s	remaining: 10m 33s
326:	learn: 0.9053700	total: 21.4s	remaining: 10m 33s
327:	learn: 0.9051943	total: 2

465:	learn: 0.8780045	total: 29.9s	remaining: 10m 10s
466:	learn: 0.8778368	total: 29.9s	remaining: 10m 10s
467:	learn: 0.8777168	total: 30s	remaining: 10m 10s
468:	learn: 0.8776200	total: 30s	remaining: 10m 9s
469:	learn: 0.8774140	total: 30.1s	remaining: 10m 9s
470:	learn: 0.8771537	total: 30.1s	remaining: 10m 9s
471:	learn: 0.8770649	total: 30.2s	remaining: 10m 8s
472:	learn: 0.8769250	total: 30.2s	remaining: 10m 8s
473:	learn: 0.8767392	total: 30.3s	remaining: 10m 8s
474:	learn: 0.8764227	total: 30.3s	remaining: 10m 7s
475:	learn: 0.8762049	total: 30.4s	remaining: 10m 7s
476:	learn: 0.8760223	total: 30.4s	remaining: 10m 7s
477:	learn: 0.8758611	total: 30.5s	remaining: 10m 6s
478:	learn: 0.8757221	total: 30.5s	remaining: 10m 6s
479:	learn: 0.8755620	total: 30.6s	remaining: 10m 6s
480:	learn: 0.8753607	total: 30.6s	remaining: 10m 5s
481:	learn: 0.8751631	total: 30.7s	remaining: 10m 5s
482:	learn: 0.8750111	total: 30.7s	remaining: 10m 5s
483:	learn: 0.8748684	total: 30.8s	remaining: 1

621:	learn: 0.8514525	total: 38.9s	remaining: 9m 46s
622:	learn: 0.8512924	total: 39s	remaining: 9m 46s
623:	learn: 0.8511876	total: 39s	remaining: 9m 46s
624:	learn: 0.8510197	total: 39.1s	remaining: 9m 46s
625:	learn: 0.8508282	total: 39.2s	remaining: 9m 46s
626:	learn: 0.8506737	total: 39.2s	remaining: 9m 46s
627:	learn: 0.8504727	total: 39.3s	remaining: 9m 46s
628:	learn: 0.8503460	total: 39.3s	remaining: 9m 46s
629:	learn: 0.8501801	total: 39.4s	remaining: 9m 46s
630:	learn: 0.8500702	total: 39.5s	remaining: 9m 46s
631:	learn: 0.8498967	total: 39.5s	remaining: 9m 46s
632:	learn: 0.8497428	total: 39.6s	remaining: 9m 46s
633:	learn: 0.8496665	total: 39.7s	remaining: 9m 46s
634:	learn: 0.8495474	total: 39.7s	remaining: 9m 46s
635:	learn: 0.8494413	total: 39.8s	remaining: 9m 46s
636:	learn: 0.8493090	total: 39.9s	remaining: 9m 46s
637:	learn: 0.8490765	total: 39.9s	remaining: 9m 46s
638:	learn: 0.8489119	total: 40s	remaining: 9m 46s
639:	learn: 0.8488279	total: 40.1s	remaining: 9m 46s

...

9285:	learn: 0.3102766	total: 8m 4s	remaining: 37.3s
9286:	learn: 0.3102490	total: 8m 4s	remaining: 37.2s
9287:	learn: 0.3102273	total: 8m 4s	remaining: 37.2s
9288:	learn: 0.3101954	total: 8m 4s	remaining: 37.1s
9289:	learn: 0.3101777	total: 8m 4s	remaining: 37.1s
9290:	learn: 0.3101598	total: 8m 4s	remaining: 37s
9291:	learn: 0.3101403	total: 8m 4s	remaining: 37s
9292:	learn: 0.3101077	total: 8m 5s	remaining: 36.9s
9293:	learn: 0.3100890	total: 8m 5s	remaining: 36.8s
9294:	learn: 0.3100613	total: 8m 5s	remaining: 36.8s
9295:	learn: 0.3100271	total: 8m 5s	remaining: 36.7s
9296:	learn: 0.3099931	total: 8m 5s	remaining: 36.7s
9297:	learn: 0.3099708	total: 8m 5s	remaining: 36.6s
9298:	learn: 0.3099324	total: 8m 5s	remaining: 36.6s
9299:	learn: 0.3099234	total: 8m 5s	remaining: 36.5s
9300:	learn: 0.3098866	total: 8m 5s	remaining: 36.5s
9301:	learn: 0.3098561	total: 8m 5s	remaining: 36.4s
9302:	learn: 0.3098352	total: 8m 5s	remaining: 36.4s
9303:	learn: 0.3098100	total: 8m 5s	remaining: 36.

9443:	learn: 0.3060360	total: 8m 12s	remaining: 29s
9444:	learn: 0.3060066	total: 8m 12s	remaining: 28.9s
9445:	learn: 0.3059886	total: 8m 12s	remaining: 28.9s
9446:	learn: 0.3059697	total: 8m 12s	remaining: 28.8s
9447:	learn: 0.3059325	total: 8m 12s	remaining: 28.8s
9448:	learn: 0.3059037	total: 8m 12s	remaining: 28.7s
9449:	learn: 0.3058791	total: 8m 12s	remaining: 28.7s
9450:	learn: 0.3058571	total: 8m 12s	remaining: 28.6s
9451:	learn: 0.3058342	total: 8m 12s	remaining: 28.6s
9452:	learn: 0.3057934	total: 8m 12s	remaining: 28.5s
9453:	learn: 0.3057666	total: 8m 12s	remaining: 28.4s
9454:	learn: 0.3057232	total: 8m 12s	remaining: 28.4s
9455:	learn: 0.3056995	total: 8m 12s	remaining: 28.3s
9456:	learn: 0.3056448	total: 8m 12s	remaining: 28.3s
9457:	learn: 0.3056158	total: 8m 12s	remaining: 28.2s
9458:	learn: 0.3055845	total: 8m 12s	remaining: 28.2s
9459:	learn: 0.3055591	total: 8m 12s	remaining: 28.1s
9460:	learn: 0.3055360	total: 8m 12s	remaining: 28.1s
9461:	learn: 0.3055172	total: 

9599:	learn: 0.3019171	total: 8m 19s	remaining: 20.8s
9600:	learn: 0.3018915	total: 8m 19s	remaining: 20.8s
9601:	learn: 0.3018788	total: 8m 19s	remaining: 20.7s
9602:	learn: 0.3018503	total: 8m 19s	remaining: 20.7s
9603:	learn: 0.3018196	total: 8m 19s	remaining: 20.6s
9604:	learn: 0.3017960	total: 8m 19s	remaining: 20.5s
9605:	learn: 0.3017721	total: 8m 19s	remaining: 20.5s
9606:	learn: 0.3017464	total: 8m 19s	remaining: 20.4s
9607:	learn: 0.3017189	total: 8m 19s	remaining: 20.4s
9608:	learn: 0.3016929	total: 8m 19s	remaining: 20.3s
9609:	learn: 0.3016694	total: 8m 19s	remaining: 20.3s
9610:	learn: 0.3016480	total: 8m 19s	remaining: 20.2s
9611:	learn: 0.3016116	total: 8m 19s	remaining: 20.2s
9612:	learn: 0.3015820	total: 8m 20s	remaining: 20.1s
9613:	learn: 0.3015374	total: 8m 20s	remaining: 20.1s
9614:	learn: 0.3015092	total: 8m 20s	remaining: 20s
9615:	learn: 0.3014878	total: 8m 20s	remaining: 20s
9616:	learn: 0.3014510	total: 8m 20s	remaining: 19.9s
9617:	learn: 0.3014160	total: 8m

9755:	learn: 0.2978539	total: 8m 26s	remaining: 12.7s
9756:	learn: 0.2978212	total: 8m 26s	remaining: 12.6s
9757:	learn: 0.2977835	total: 8m 26s	remaining: 12.6s
9758:	learn: 0.2977682	total: 8m 26s	remaining: 12.5s
9759:	learn: 0.2977421	total: 8m 26s	remaining: 12.5s
9760:	learn: 0.2977195	total: 8m 27s	remaining: 12.4s
9761:	learn: 0.2976934	total: 8m 27s	remaining: 12.4s
9762:	learn: 0.2976575	total: 8m 27s	remaining: 12.3s
9763:	learn: 0.2976318	total: 8m 27s	remaining: 12.3s
9764:	learn: 0.2976018	total: 8m 27s	remaining: 12.2s
9765:	learn: 0.2975716	total: 8m 27s	remaining: 12.2s
9766:	learn: 0.2975466	total: 8m 27s	remaining: 12.1s
9767:	learn: 0.2975326	total: 8m 27s	remaining: 12.1s
9768:	learn: 0.2975099	total: 8m 27s	remaining: 12s
9769:	learn: 0.2974830	total: 8m 27s	remaining: 11.9s
9770:	learn: 0.2974630	total: 8m 27s	remaining: 11.9s
9771:	learn: 0.2974345	total: 8m 27s	remaining: 11.8s
9772:	learn: 0.2974114	total: 8m 27s	remaining: 11.8s
9773:	learn: 0.2973947	total: 

9912:	learn: 0.2938332	total: 8m 34s	remaining: 4.51s
9913:	learn: 0.2938009	total: 8m 34s	remaining: 4.46s
9914:	learn: 0.2937900	total: 8m 34s	remaining: 4.41s
9915:	learn: 0.2937778	total: 8m 34s	remaining: 4.36s
9916:	learn: 0.2937520	total: 8m 34s	remaining: 4.3s
9917:	learn: 0.2937240	total: 8m 34s	remaining: 4.25s
9918:	learn: 0.2937042	total: 8m 34s	remaining: 4.2s
9919:	learn: 0.2936824	total: 8m 34s	remaining: 4.15s
9920:	learn: 0.2936603	total: 8m 34s	remaining: 4.1s
9921:	learn: 0.2936253	total: 8m 34s	remaining: 4.04s
9922:	learn: 0.2935876	total: 8m 34s	remaining: 3.99s
9923:	learn: 0.2935721	total: 8m 34s	remaining: 3.94s
9924:	learn: 0.2935566	total: 8m 34s	remaining: 3.89s
9925:	learn: 0.2935385	total: 8m 34s	remaining: 3.84s
9926:	learn: 0.2934964	total: 8m 34s	remaining: 3.79s
9927:	learn: 0.2934503	total: 8m 34s	remaining: 3.73s
9928:	learn: 0.2934317	total: 8m 34s	remaining: 3.68s
9929:	learn: 0.2933881	total: 8m 34s	remaining: 3.63s
9930:	learn: 0.2933510	total: 8

In [36]:
headers = data_all.columns.values.tolist()
for i in range(len(arousal_model.feature_importances_)):
    print(headers[i],' :', arousal_model.feature_importances_[i])

channel_AF3_mean  : 2.2295640782942643
channel_AF3_first_der  : 1.041048175561301
channel_AF3_second_der  : 0.633739213065568
channel_AF3_median  : 0.7643988964770921
channel_AF3_freq_beta  : 1.1053428885183865
channel_AF3_freq_gamma  : 1.77333567914248
channel_F3_mean  : 1.4858671581212852
channel_F3_first_der  : 0.539982324582639
channel_F3_second_der  : 0.3849836906767161
channel_F3_median  : 0.6918686567268322
channel_F3_freq_beta  : 0.6768669678378652
channel_F3_freq_gamma  : 0.8447473517859507
channel_F7_mean  : 1.8648433015150554
channel_F7_first_der  : 0.5885858814711877
channel_F7_second_der  : 0.3203217418117568
channel_F7_median  : 0.8670886880627231
channel_F7_freq_beta  : 0.7256551966105322
channel_F7_freq_gamma  : 0.9974644613041772
channel_FC5_mean  : 2.0012584949455148
channel_FC5_first_der  : 0.8873114969317067
channel_FC5_second_der  : 0.5240758382936442
channel_FC5_median  : 0.7682106266712748
channel_FC5_freq_beta  : 0.840494218377793
channel_FC5_freq_gamma  : 0.908

In [41]:
arousal_model.score(X, y)

['channel_AF3_mean',
 'channel_AF3_first_der',
 'channel_AF3_second_der',
 'channel_AF3_median',
 'channel_AF3_freq_beta',
 'channel_AF3_freq_gamma',
 'channel_F3_mean',
 'channel_F3_first_der',
 'channel_F3_second_der',
 'channel_F3_median',
 'channel_F3_freq_beta',
 'channel_F3_freq_gamma',
 'channel_F7_mean',
 'channel_F7_first_der',
 'channel_F7_second_der',
 'channel_F7_median',
 'channel_F7_freq_beta',
 'channel_F7_freq_gamma',
 'channel_FC5_mean',
 'channel_FC5_first_der',
 'channel_FC5_second_der',
 'channel_FC5_median',
 'channel_FC5_freq_beta',
 'channel_FC5_freq_gamma',
 'channel_T7_mean',
 'channel_T7_first_der',
 'channel_T7_second_der',
 'channel_T7_median',
 'channel_T7_freq_beta',
 'channel_T7_freq_gamma',
 'channel_P7_mean',
 'channel_P7_first_der',
 'channel_P7_second_der',
 'channel_P7_median',
 'channel_P7_freq_beta',
 'channel_P7_freq_gamma',
 'channel_O1_mean',
 'channel_O1_first_der',
 'channel_O1_second_der',
 'channel_O1_median',
 'channel_O1_freq_beta',
 'chan

In [76]:
import pickle as pkl

pkl.dump(arousal_model, open("arousal_removed_eyes.pkl", "wb"))


In [77]:
dominance_model = train_model('dominance', data_all, cat_dominance_model)
pkl.dump(dominance_model, open("dominance_removed_eyes.pkl", "wb"))

Learning rate set to 0.012876
0:	learn: 1.0958679	total: 113ms	remaining: 18m 49s
1:	learn: 1.0934785	total: 223ms	remaining: 18m 34s
2:	learn: 1.0909030	total: 319ms	remaining: 17m 43s
3:	learn: 1.0882515	total: 412ms	remaining: 17m 9s
4:	learn: 1.0853484	total: 512ms	remaining: 17m 3s
5:	learn: 1.0829144	total: 605ms	remaining: 16m 48s
6:	learn: 1.0804449	total: 703ms	remaining: 16m 42s
7:	learn: 1.0775012	total: 798ms	remaining: 16m 37s
8:	learn: 1.0753319	total: 895ms	remaining: 16m 33s
9:	learn: 1.0727160	total: 1.01s	remaining: 16m 47s
10:	learn: 1.0702974	total: 1.1s	remaining: 16m 43s
11:	learn: 1.0682454	total: 1.21s	remaining: 16m 45s
12:	learn: 1.0656736	total: 1.31s	remaining: 16m 45s
13:	learn: 1.0636410	total: 1.4s	remaining: 16m 37s
14:	learn: 1.0613855	total: 1.5s	remaining: 16m 36s
15:	learn: 1.0595906	total: 1.59s	remaining: 16m 31s
16:	learn: 1.0572248	total: 1.69s	remaining: 16m 32s
17:	learn: 1.0549892	total: 1.79s	remaining: 16m 31s
18:	learn: 1.0530698	total: 1.8

156:	learn: 0.9242515	total: 15.7s	remaining: 16m 22s
157:	learn: 0.9237826	total: 15.8s	remaining: 16m 22s
158:	learn: 0.9231579	total: 15.9s	remaining: 16m 22s
159:	learn: 0.9228427	total: 16s	remaining: 16m 21s
160:	learn: 0.9225385	total: 16s	remaining: 16m 20s
161:	learn: 0.9220684	total: 16.1s	remaining: 16m 19s
162:	learn: 0.9215557	total: 16.2s	remaining: 16m 19s
163:	learn: 0.9211976	total: 16.3s	remaining: 16m 19s
164:	learn: 0.9208840	total: 16.4s	remaining: 16m 18s
165:	learn: 0.9204433	total: 16.5s	remaining: 16m 17s
166:	learn: 0.9199886	total: 16.6s	remaining: 16m 17s
167:	learn: 0.9196490	total: 16.7s	remaining: 16m 16s
168:	learn: 0.9192774	total: 16.8s	remaining: 16m 16s
169:	learn: 0.9188087	total: 16.9s	remaining: 16m 15s
170:	learn: 0.9184424	total: 17s	remaining: 16m 15s
171:	learn: 0.9180148	total: 17.1s	remaining: 16m 14s
172:	learn: 0.9177397	total: 17.2s	remaining: 16m 14s
173:	learn: 0.9172956	total: 17.3s	remaining: 16m 14s
174:	learn: 0.9168057	total: 17.3s

309:	learn: 0.8762064	total: 31.4s	remaining: 16m 22s
310:	learn: 0.8759854	total: 31.5s	remaining: 16m 22s
311:	learn: 0.8758555	total: 31.6s	remaining: 16m 22s
312:	learn: 0.8756398	total: 31.8s	remaining: 16m 22s
313:	learn: 0.8752888	total: 31.9s	remaining: 16m 22s
314:	learn: 0.8750914	total: 32s	remaining: 16m 22s
315:	learn: 0.8748627	total: 32.1s	remaining: 16m 22s
316:	learn: 0.8747608	total: 32.1s	remaining: 16m 21s
317:	learn: 0.8745473	total: 32.2s	remaining: 16m 21s
318:	learn: 0.8743147	total: 32.3s	remaining: 16m 21s
319:	learn: 0.8740838	total: 32.4s	remaining: 16m 20s
320:	learn: 0.8738680	total: 32.5s	remaining: 16m 20s
321:	learn: 0.8736587	total: 32.6s	remaining: 16m 20s
322:	learn: 0.8734334	total: 32.7s	remaining: 16m 20s
323:	learn: 0.8732580	total: 32.8s	remaining: 16m 20s
324:	learn: 0.8729549	total: 32.9s	remaining: 16m 20s
325:	learn: 0.8727611	total: 33s	remaining: 16m 20s
326:	learn: 0.8725433	total: 33.2s	remaining: 16m 20s
327:	learn: 0.8722952	total: 33.

464:	learn: 0.8452281	total: 46.9s	remaining: 16m 1s
465:	learn: 0.8451394	total: 47s	remaining: 16m 1s
466:	learn: 0.8449093	total: 47.1s	remaining: 16m 1s
467:	learn: 0.8447675	total: 47.2s	remaining: 16m 1s
468:	learn: 0.8446338	total: 47.3s	remaining: 16m
469:	learn: 0.8444547	total: 47.4s	remaining: 16m
470:	learn: 0.8442411	total: 47.5s	remaining: 16m
471:	learn: 0.8440971	total: 47.6s	remaining: 16m
472:	learn: 0.8439071	total: 47.7s	remaining: 16m
473:	learn: 0.8437332	total: 47.8s	remaining: 16m
474:	learn: 0.8434384	total: 47.9s	remaining: 16m 1s
475:	learn: 0.8432853	total: 48s	remaining: 16m 1s
476:	learn: 0.8431370	total: 48.2s	remaining: 16m 1s
477:	learn: 0.8429214	total: 48.3s	remaining: 16m 1s
478:	learn: 0.8426653	total: 48.4s	remaining: 16m 1s
479:	learn: 0.8424987	total: 48.5s	remaining: 16m 1s
480:	learn: 0.8423054	total: 48.5s	remaining: 16m
481:	learn: 0.8419864	total: 48.6s	remaining: 16m
482:	learn: 0.8417397	total: 48.7s	remaining: 16m
483:	learn: 0.8415522	to

618:	learn: 0.8188597	total: 1m 1s	remaining: 15m 38s
619:	learn: 0.8187522	total: 1m 1s	remaining: 15m 37s
620:	learn: 0.8186297	total: 1m 2s	remaining: 15m 37s
621:	learn: 0.8184802	total: 1m 2s	remaining: 15m 37s
622:	learn: 0.8183060	total: 1m 2s	remaining: 15m 37s
623:	learn: 0.8181870	total: 1m 2s	remaining: 15m 36s
624:	learn: 0.8179827	total: 1m 2s	remaining: 15m 36s
625:	learn: 0.8178010	total: 1m 2s	remaining: 15m 36s
626:	learn: 0.8176006	total: 1m 2s	remaining: 15m 36s
627:	learn: 0.8173609	total: 1m 2s	remaining: 15m 36s
628:	learn: 0.8171852	total: 1m 2s	remaining: 15m 36s
629:	learn: 0.8170130	total: 1m 2s	remaining: 15m 35s
630:	learn: 0.8168596	total: 1m 3s	remaining: 15m 35s
631:	learn: 0.8167867	total: 1m 3s	remaining: 15m 35s
632:	learn: 0.8166240	total: 1m 3s	remaining: 15m 35s
633:	learn: 0.8164340	total: 1m 3s	remaining: 15m 35s
634:	learn: 0.8162981	total: 1m 3s	remaining: 15m 35s
635:	learn: 0.8162357	total: 1m 3s	remaining: 15m 34s
636:	learn: 0.8161386	total:

...

9282:	learn: 0.2744167	total: 15m 55s	remaining: 1m 13s
9283:	learn: 0.2743986	total: 15m 55s	remaining: 1m 13s
9284:	learn: 0.2743634	total: 15m 56s	remaining: 1m 13s
9285:	learn: 0.2743434	total: 15m 56s	remaining: 1m 13s
9286:	learn: 0.2743255	total: 15m 56s	remaining: 1m 13s
9287:	learn: 0.2742948	total: 15m 56s	remaining: 1m 13s
9288:	learn: 0.2742505	total: 15m 56s	remaining: 1m 13s
9289:	learn: 0.2742256	total: 15m 56s	remaining: 1m 13s
9290:	learn: 0.2742049	total: 15m 56s	remaining: 1m 13s
9291:	learn: 0.2741756	total: 15m 56s	remaining: 1m 12s
9292:	learn: 0.2741481	total: 15m 56s	remaining: 1m 12s
9293:	learn: 0.2741279	total: 15m 56s	remaining: 1m 12s
9294:	learn: 0.2741057	total: 15m 57s	remaining: 1m 12s
9295:	learn: 0.2740759	total: 15m 57s	remaining: 1m 12s
9296:	learn: 0.2740278	total: 15m 57s	remaining: 1m 12s
9297:	learn: 0.2740008	total: 15m 57s	remaining: 1m 12s
9298:	learn: 0.2739684	total: 15m 57s	remaining: 1m 12s
9299:	learn: 0.2739374	total: 15m 57s	remaining:

9434:	learn: 0.2703798	total: 16m 11s	remaining: 58.2s
9435:	learn: 0.2703522	total: 16m 11s	remaining: 58.1s
9436:	learn: 0.2703242	total: 16m 11s	remaining: 58s
9437:	learn: 0.2702987	total: 16m 11s	remaining: 57.9s
9438:	learn: 0.2702741	total: 16m 12s	remaining: 57.8s
9439:	learn: 0.2702324	total: 16m 12s	remaining: 57.7s
9440:	learn: 0.2702122	total: 16m 12s	remaining: 57.6s
9441:	learn: 0.2701910	total: 16m 12s	remaining: 57.5s
9442:	learn: 0.2701626	total: 16m 12s	remaining: 57.4s
9443:	learn: 0.2701377	total: 16m 12s	remaining: 57.3s
9444:	learn: 0.2701054	total: 16m 12s	remaining: 57.2s
9445:	learn: 0.2700809	total: 16m 12s	remaining: 57.1s
9446:	learn: 0.2700586	total: 16m 12s	remaining: 56.9s
9447:	learn: 0.2700286	total: 16m 12s	remaining: 56.8s
9448:	learn: 0.2700046	total: 16m 13s	remaining: 56.7s
9449:	learn: 0.2699571	total: 16m 13s	remaining: 56.6s
9450:	learn: 0.2699362	total: 16m 13s	remaining: 56.5s
9451:	learn: 0.2699105	total: 16m 13s	remaining: 56.4s
9452:	learn:

9585:	learn: 0.2663082	total: 16m 27s	remaining: 42.7s
9586:	learn: 0.2662814	total: 16m 27s	remaining: 42.5s
9587:	learn: 0.2662618	total: 16m 27s	remaining: 42.4s
9588:	learn: 0.2662436	total: 16m 27s	remaining: 42.3s
9589:	learn: 0.2662058	total: 16m 28s	remaining: 42.2s
9590:	learn: 0.2661867	total: 16m 28s	remaining: 42.1s
9591:	learn: 0.2661439	total: 16m 28s	remaining: 42s
9592:	learn: 0.2661151	total: 16m 28s	remaining: 41.9s
9593:	learn: 0.2660846	total: 16m 28s	remaining: 41.8s
9594:	learn: 0.2660667	total: 16m 28s	remaining: 41.7s
9595:	learn: 0.2660508	total: 16m 28s	remaining: 41.6s
9596:	learn: 0.2660293	total: 16m 28s	remaining: 41.5s
9597:	learn: 0.2660026	total: 16m 28s	remaining: 41.4s
9598:	learn: 0.2659923	total: 16m 29s	remaining: 41.3s
9599:	learn: 0.2659733	total: 16m 29s	remaining: 41.2s
9600:	learn: 0.2659617	total: 16m 29s	remaining: 41.1s
9601:	learn: 0.2659393	total: 16m 29s	remaining: 41s
9602:	learn: 0.2659222	total: 16m 29s	remaining: 40.9s
9603:	learn: 0

9735:	learn: 0.2626325	total: 16m 43s	remaining: 27.2s
9736:	learn: 0.2626017	total: 16m 43s	remaining: 27.1s
9737:	learn: 0.2625743	total: 16m 43s	remaining: 27s
9738:	learn: 0.2625477	total: 16m 43s	remaining: 26.9s
9739:	learn: 0.2625206	total: 16m 44s	remaining: 26.8s
9740:	learn: 0.2624933	total: 16m 44s	remaining: 26.7s
9741:	learn: 0.2624682	total: 16m 44s	remaining: 26.6s
9742:	learn: 0.2624442	total: 16m 44s	remaining: 26.5s
9743:	learn: 0.2624185	total: 16m 44s	remaining: 26.4s
9744:	learn: 0.2623926	total: 16m 44s	remaining: 26.3s
9745:	learn: 0.2623592	total: 16m 44s	remaining: 26.2s
9746:	learn: 0.2623437	total: 16m 44s	remaining: 26.1s
9747:	learn: 0.2623241	total: 16m 45s	remaining: 26s
9748:	learn: 0.2622994	total: 16m 45s	remaining: 25.9s
9749:	learn: 0.2622753	total: 16m 45s	remaining: 25.8s
9750:	learn: 0.2622443	total: 16m 45s	remaining: 25.7s
9751:	learn: 0.2622104	total: 16m 45s	remaining: 25.6s
9752:	learn: 0.2621942	total: 16m 45s	remaining: 25.5s
9753:	learn: 0

9887:	learn: 0.2588368	total: 17m 1s	remaining: 11.6s
9888:	learn: 0.2588108	total: 17m 1s	remaining: 11.5s
9889:	learn: 0.2587900	total: 17m 1s	remaining: 11.4s
9890:	learn: 0.2587598	total: 17m 1s	remaining: 11.3s
9891:	learn: 0.2587369	total: 17m 2s	remaining: 11.2s
9892:	learn: 0.2587073	total: 17m 2s	remaining: 11.1s
9893:	learn: 0.2586620	total: 17m 2s	remaining: 11s
9894:	learn: 0.2586413	total: 17m 2s	remaining: 10.8s
9895:	learn: 0.2586124	total: 17m 2s	remaining: 10.7s
9896:	learn: 0.2585978	total: 17m 2s	remaining: 10.6s
9897:	learn: 0.2585867	total: 17m 2s	remaining: 10.5s
9898:	learn: 0.2585598	total: 17m 2s	remaining: 10.4s
9899:	learn: 0.2585408	total: 17m 2s	remaining: 10.3s
9900:	learn: 0.2585234	total: 17m 3s	remaining: 10.2s
9901:	learn: 0.2585044	total: 17m 3s	remaining: 10.1s
9902:	learn: 0.2584849	total: 17m 3s	remaining: 10s
9903:	learn: 0.2584722	total: 17m 3s	remaining: 9.92s
9904:	learn: 0.2584566	total: 17m 3s	remaining: 9.82s
9905:	learn: 0.2584326	total: 17

In [78]:
valence_model = train_model('valence', data_all, cat_valence_model)
pkl.dump(valence_model, open("valence_removed_eyes.pkl", "wb"))

Learning rate set to 0.012876
0:	learn: 1.0956527	total: 109ms	remaining: 18m 10s
1:	learn: 1.0928755	total: 214ms	remaining: 17m 50s
2:	learn: 1.0897359	total: 317ms	remaining: 17m 37s
3:	learn: 1.0867512	total: 416ms	remaining: 17m 18s
4:	learn: 1.0839641	total: 517ms	remaining: 17m 12s
5:	learn: 1.0812460	total: 612ms	remaining: 16m 59s
6:	learn: 1.0786868	total: 712ms	remaining: 16m 56s
7:	learn: 1.0760775	total: 813ms	remaining: 16m 55s
8:	learn: 1.0735873	total: 918ms	remaining: 16m 58s
9:	learn: 1.0708583	total: 1.02s	remaining: 17m 4s
10:	learn: 1.0685354	total: 1.12s	remaining: 16m 57s
11:	learn: 1.0659088	total: 1.22s	remaining: 16m 57s
12:	learn: 1.0633131	total: 1.32s	remaining: 16m 58s
13:	learn: 1.0608639	total: 1.43s	remaining: 16m 56s
14:	learn: 1.0586781	total: 1.53s	remaining: 16m 57s
15:	learn: 1.0562012	total: 1.63s	remaining: 16m 54s
16:	learn: 1.0538735	total: 1.72s	remaining: 16m 52s
17:	learn: 1.0513259	total: 1.82s	remaining: 16m 50s
18:	learn: 1.0491616	total:

156:	learn: 0.9249054	total: 16.1s	remaining: 16m 49s
157:	learn: 0.9245497	total: 16.2s	remaining: 16m 48s
158:	learn: 0.9240328	total: 16.3s	remaining: 16m 48s
159:	learn: 0.9237856	total: 16.4s	remaining: 16m 47s
160:	learn: 0.9232840	total: 16.5s	remaining: 16m 47s
161:	learn: 0.9228699	total: 16.6s	remaining: 16m 47s
162:	learn: 0.9223594	total: 16.7s	remaining: 16m 46s
163:	learn: 0.9220397	total: 16.8s	remaining: 16m 46s
164:	learn: 0.9216621	total: 16.9s	remaining: 16m 46s
165:	learn: 0.9212426	total: 17s	remaining: 16m 45s
166:	learn: 0.9208641	total: 17.1s	remaining: 16m 45s
167:	learn: 0.9204771	total: 17.2s	remaining: 16m 46s
168:	learn: 0.9201507	total: 17.3s	remaining: 16m 46s
169:	learn: 0.9197174	total: 17.4s	remaining: 16m 47s
170:	learn: 0.9192019	total: 17.5s	remaining: 16m 47s
171:	learn: 0.9188310	total: 17.6s	remaining: 16m 47s
172:	learn: 0.9184722	total: 17.7s	remaining: 16m 46s
173:	learn: 0.9178794	total: 17.8s	remaining: 16m 46s
174:	learn: 0.9174850	total: 1

309:	learn: 0.8819472	total: 31.9s	remaining: 16m 35s
310:	learn: 0.8817796	total: 32s	remaining: 16m 35s
311:	learn: 0.8816473	total: 32.1s	remaining: 16m 35s
312:	learn: 0.8814349	total: 32.2s	remaining: 16m 35s
313:	learn: 0.8812382	total: 32.3s	remaining: 16m 34s
314:	learn: 0.8810257	total: 32.4s	remaining: 16m 34s
315:	learn: 0.8806120	total: 32.5s	remaining: 16m 34s
316:	learn: 0.8804034	total: 32.6s	remaining: 16m 34s
317:	learn: 0.8802140	total: 32.7s	remaining: 16m 34s
318:	learn: 0.8799802	total: 32.8s	remaining: 16m 34s
319:	learn: 0.8797194	total: 32.9s	remaining: 16m 34s
320:	learn: 0.8794991	total: 33s	remaining: 16m 34s
321:	learn: 0.8793428	total: 33.1s	remaining: 16m 33s
322:	learn: 0.8791504	total: 33.2s	remaining: 16m 33s
323:	learn: 0.8790188	total: 33.3s	remaining: 16m 33s
324:	learn: 0.8788333	total: 33.4s	remaining: 16m 33s
325:	learn: 0.8786602	total: 33.5s	remaining: 16m 33s
326:	learn: 0.8784655	total: 33.6s	remaining: 16m 33s
327:	learn: 0.8782842	total: 33.

464:	learn: 0.8518770	total: 47.5s	remaining: 16m 13s
465:	learn: 0.8517601	total: 47.6s	remaining: 16m 13s
466:	learn: 0.8515591	total: 47.7s	remaining: 16m 13s
467:	learn: 0.8514169	total: 47.8s	remaining: 16m 13s
468:	learn: 0.8512525	total: 47.9s	remaining: 16m 13s
469:	learn: 0.8511617	total: 48s	remaining: 16m 13s
470:	learn: 0.8508650	total: 48.1s	remaining: 16m 12s
471:	learn: 0.8507469	total: 48.2s	remaining: 16m 12s
472:	learn: 0.8506289	total: 48.3s	remaining: 16m 12s
473:	learn: 0.8504784	total: 48.4s	remaining: 16m 12s
474:	learn: 0.8502781	total: 48.5s	remaining: 16m 12s
475:	learn: 0.8501638	total: 48.6s	remaining: 16m 11s
476:	learn: 0.8500341	total: 48.7s	remaining: 16m 11s
477:	learn: 0.8498084	total: 48.8s	remaining: 16m 11s
478:	learn: 0.8496366	total: 48.9s	remaining: 16m 11s
479:	learn: 0.8494941	total: 49s	remaining: 16m 11s
480:	learn: 0.8491230	total: 49.1s	remaining: 16m 11s
481:	learn: 0.8489620	total: 49.2s	remaining: 16m 11s
482:	learn: 0.8488424	total: 49.

619:	learn: 0.8273888	total: 1m 2s	remaining: 15m 51s
620:	learn: 0.8271704	total: 1m 2s	remaining: 15m 51s
621:	learn: 0.8269289	total: 1m 3s	remaining: 15m 51s
622:	learn: 0.8267898	total: 1m 3s	remaining: 15m 50s
623:	learn: 0.8266231	total: 1m 3s	remaining: 15m 50s
624:	learn: 0.8264860	total: 1m 3s	remaining: 15m 50s
625:	learn: 0.8263583	total: 1m 3s	remaining: 15m 50s
626:	learn: 0.8262657	total: 1m 3s	remaining: 15m 50s
627:	learn: 0.8261334	total: 1m 3s	remaining: 15m 50s
628:	learn: 0.8260590	total: 1m 3s	remaining: 15m 50s
629:	learn: 0.8258913	total: 1m 3s	remaining: 15m 50s
630:	learn: 0.8257365	total: 1m 3s	remaining: 15m 50s
631:	learn: 0.8256204	total: 1m 4s	remaining: 15m 49s
632:	learn: 0.8254400	total: 1m 4s	remaining: 15m 49s
633:	learn: 0.8252787	total: 1m 4s	remaining: 15m 49s
634:	learn: 0.8250861	total: 1m 4s	remaining: 15m 49s
635:	learn: 0.8249818	total: 1m 4s	remaining: 15m 49s
636:	learn: 0.8248508	total: 1m 4s	remaining: 15m 49s
637:	learn: 0.8247004	total:

...

9274:	learn: 0.2791919	total: 16m 11s	remaining: 1m 15s
9275:	learn: 0.2791646	total: 16m 11s	remaining: 1m 15s
9276:	learn: 0.2791454	total: 16m 12s	remaining: 1m 15s
9277:	learn: 0.2791025	total: 16m 12s	remaining: 1m 15s
9278:	learn: 0.2790767	total: 16m 12s	remaining: 1m 15s
9279:	learn: 0.2790452	total: 16m 12s	remaining: 1m 15s
9280:	learn: 0.2790282	total: 16m 12s	remaining: 1m 15s
9281:	learn: 0.2789994	total: 16m 12s	remaining: 1m 15s
9282:	learn: 0.2789632	total: 16m 12s	remaining: 1m 15s
9283:	learn: 0.2789439	total: 16m 12s	remaining: 1m 15s
9284:	learn: 0.2788918	total: 16m 12s	remaining: 1m 14s
9285:	learn: 0.2788645	total: 16m 12s	remaining: 1m 14s
9286:	learn: 0.2788351	total: 16m 13s	remaining: 1m 14s
9287:	learn: 0.2788102	total: 16m 13s	remaining: 1m 14s
9288:	learn: 0.2787857	total: 16m 13s	remaining: 1m 14s
9289:	learn: 0.2787701	total: 16m 13s	remaining: 1m 14s
9290:	learn: 0.2787530	total: 16m 13s	remaining: 1m 14s
9291:	learn: 0.2787254	total: 16m 13s	remaining:

9423:	learn: 0.2753594	total: 16m 27s	remaining: 1m
9424:	learn: 0.2753413	total: 16m 27s	remaining: 1m
9425:	learn: 0.2753060	total: 16m 27s	remaining: 1m
9426:	learn: 0.2752835	total: 16m 27s	remaining: 1m
9427:	learn: 0.2752703	total: 16m 27s	remaining: 59.9s
9428:	learn: 0.2752506	total: 16m 27s	remaining: 59.8s
9429:	learn: 0.2752184	total: 16m 27s	remaining: 59.7s
9430:	learn: 0.2751904	total: 16m 27s	remaining: 59.6s
9431:	learn: 0.2751557	total: 16m 28s	remaining: 59.5s
9432:	learn: 0.2751370	total: 16m 28s	remaining: 59.4s
9433:	learn: 0.2751136	total: 16m 28s	remaining: 59.3s
9434:	learn: 0.2750963	total: 16m 28s	remaining: 59.2s
9435:	learn: 0.2750661	total: 16m 28s	remaining: 59.1s
9436:	learn: 0.2750064	total: 16m 28s	remaining: 59s
9437:	learn: 0.2749871	total: 16m 28s	remaining: 58.9s
9438:	learn: 0.2749595	total: 16m 28s	remaining: 58.8s
9439:	learn: 0.2749212	total: 16m 28s	remaining: 58.7s
9440:	learn: 0.2748986	total: 16m 29s	remaining: 58.6s
9441:	learn: 0.2748733	t

9573:	learn: 0.2715170	total: 16m 43s	remaining: 44.7s
9574:	learn: 0.2715009	total: 16m 43s	remaining: 44.5s
9575:	learn: 0.2714837	total: 16m 43s	remaining: 44.4s
9576:	learn: 0.2714603	total: 16m 43s	remaining: 44.3s
9577:	learn: 0.2714360	total: 16m 43s	remaining: 44.2s
9578:	learn: 0.2713981	total: 16m 44s	remaining: 44.1s
9579:	learn: 0.2713588	total: 16m 44s	remaining: 44s
9580:	learn: 0.2713360	total: 16m 44s	remaining: 43.9s
9581:	learn: 0.2713004	total: 16m 44s	remaining: 43.8s
9582:	learn: 0.2712742	total: 16m 44s	remaining: 43.7s
9583:	learn: 0.2712294	total: 16m 44s	remaining: 43.6s
9584:	learn: 0.2711944	total: 16m 44s	remaining: 43.5s
9585:	learn: 0.2711591	total: 16m 44s	remaining: 43.4s
9586:	learn: 0.2711332	total: 16m 44s	remaining: 43.3s
9587:	learn: 0.2711183	total: 16m 45s	remaining: 43.2s
9588:	learn: 0.2710762	total: 16m 45s	remaining: 43.1s
9589:	learn: 0.2710637	total: 16m 45s	remaining: 43s
9590:	learn: 0.2710335	total: 16m 45s	remaining: 42.9s
9591:	learn: 0

9723:	learn: 0.2677599	total: 16m 59s	remaining: 28.9s
9724:	learn: 0.2677188	total: 16m 59s	remaining: 28.8s
9725:	learn: 0.2677034	total: 16m 59s	remaining: 28.7s
9726:	learn: 0.2676836	total: 16m 59s	remaining: 28.6s
9727:	learn: 0.2676629	total: 16m 59s	remaining: 28.5s
9728:	learn: 0.2676373	total: 16m 59s	remaining: 28.4s
9729:	learn: 0.2676210	total: 16m 59s	remaining: 28.3s
9730:	learn: 0.2675767	total: 17m	remaining: 28.2s
9731:	learn: 0.2675552	total: 17m	remaining: 28.1s
9732:	learn: 0.2675252	total: 17m	remaining: 28s
9733:	learn: 0.2675047	total: 17m	remaining: 27.9s
9734:	learn: 0.2674905	total: 17m	remaining: 27.8s
9735:	learn: 0.2674687	total: 17m	remaining: 27.7s
9736:	learn: 0.2674538	total: 17m	remaining: 27.6s
9737:	learn: 0.2674226	total: 17m	remaining: 27.5s
9738:	learn: 0.2673944	total: 17m	remaining: 27.4s
9739:	learn: 0.2673721	total: 17m 1s	remaining: 27.3s
9740:	learn: 0.2673526	total: 17m 1s	remaining: 27.1s
9741:	learn: 0.2673253	total: 17m 1s	remaining: 27

9875:	learn: 0.2639968	total: 17m 16s	remaining: 13s
9876:	learn: 0.2639643	total: 17m 16s	remaining: 12.9s
9877:	learn: 0.2639485	total: 17m 16s	remaining: 12.8s
9878:	learn: 0.2639304	total: 17m 16s	remaining: 12.7s
9879:	learn: 0.2639157	total: 17m 16s	remaining: 12.6s
9880:	learn: 0.2638873	total: 17m 16s	remaining: 12.5s
9881:	learn: 0.2638579	total: 17m 16s	remaining: 12.4s
9882:	learn: 0.2638426	total: 17m 17s	remaining: 12.3s
9883:	learn: 0.2638099	total: 17m 17s	remaining: 12.2s
9884:	learn: 0.2637897	total: 17m 17s	remaining: 12.1s
9885:	learn: 0.2637683	total: 17m 17s	remaining: 12s
9886:	learn: 0.2637506	total: 17m 17s	remaining: 11.9s
9887:	learn: 0.2637325	total: 17m 17s	remaining: 11.8s
9888:	learn: 0.2637041	total: 17m 17s	remaining: 11.6s
9889:	learn: 0.2636883	total: 17m 17s	remaining: 11.5s
9890:	learn: 0.2636669	total: 17m 17s	remaining: 11.4s
9891:	learn: 0.2636356	total: 17m 17s	remaining: 11.3s
9892:	learn: 0.2636199	total: 17m 18s	remaining: 11.2s
9893:	learn: 0

In [5]:
with open('arousal.pkl', 'rb') as f:
    model_arousal = pkl.load(f)
with open('dominance.pkl', 'rb') as f:
    model_dominance = pkl.load(f)
with open('valence.pkl', 'rb') as f:
    model_valence = pkl.load(f)

In [23]:
f = open('data_preprocessed_python/s01.dat', 'rb')
data = pickle.loads(f.read(), encoding='latin1')

TypeError: expected sequence object with len >= 0 or a single integer